In [1]:
import textract
import spacy
import pandas as pd

# 1. Extracting text 

In [2]:
# Extracting text using tika-python
text = textract.process('sample.pdf').decode()
print(text[:211])

Science
Science (from the Latin word scientia, meaning
"knowledge")[1] is a systematic enterprise that builds and
organizes knowledge in the form of testable explanations
and predictions about the universe.[2][3


# 2. Processing the text using spaCy

In [3]:
# Loading the spaCy 'small' model
nlp = spacy.load("en_core_web_sm")

doc = nlp(text)  # applying the spaCy NLP pipeline

# Obtaining entity information
ent_text = []
ent_labels = []
ent_sentences = []
for entity in doc.ents:
    ent_text.append(entity.text)  # the entity's text
    ent_labels.append(entity.label_)  # the type of entity (PERSON, ORG, GPE, etc)
    ent_sentences.append(entity.sent.text.replace('\n',' '))  # some context of the entity

len(ent_labels),len(ent_text),len(ent_sentences),len(doc.ents)

(2653, 2653, 2653, 2653)

# 3. Exporting results to an excel file

In [4]:
#Saving the information as a dataframe
text_data = pd.DataFrame({'Entity': ent_text,'Type': ent_labels,'Context':ent_sentences})
text_data.head()

,Entity,Type,Context
0,Latin,NORP,"Science Science (from the Latin word scientia,..."
1,Egypt,GPE,The earliest roots of science can be traced to...
2,Mesopotamia,LOC,The earliest roots of science can be traced to...
3,around 3500 to 3000,CARDINAL,The earliest roots of science can be traced to...
4,Greek,NORP,"Their contributions to mathematics, astronomy,..."


In [5]:
# Getting brief descriptions of each entity type

labels = text_data.Type.unique()
ent_descriptions = [spacy.explain(_label) for _label in labels]
Descriptions = pd.DataFrame({"Type": labels, "Description": ent_descriptions})
Descriptions

,Type,Description
0,NORP,Nationalities or religious or political groups
1,GPE,"Countries, cities, states"
2,LOC,"Non-GPE locations, mountain ranges, bodies of ..."
3,CARDINAL,Numerals that do not fall under another type
4,ORG,"Companies, agencies, institutions, etc."
5,DATE,Absolute or relative dates or periods
6,PERSON,"People, including fictional"
7,ORDINAL,"""first"", ""second"", etc."
8,WORK_OF_ART,"Titles of books, songs, etc."
9,PRODUCT,"Objects, vehicles, foods, etc. (not services)"


In [6]:
with pd.ExcelWriter('text-mining.xlsx') as writer:
    Descriptions.to_excel(writer, sheet_name="DEFINITIONS", index=False)
    for ent_name, data in text_data.groupby('Type'):
        data[["Entity","Context"]].to_excel(writer, sheet_name=ent_name, 
                                            index=False)